In [1]:
from data_rnn import load_ndfa, load_brackets
from data_prep import pad_and_convert
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [3]:
x_train_ndfa, (i2w_ndfa, w2i_ndfa) = load_ndfa(n=1000)
x_train_brackets, (i2w_brackets, w2i_brackets) = load_brackets(n=150_000)

In [73]:
print(''.join([i2w_ndfa[i] for i in  x_train_ndfa[10_000]]) )

IndexError: list index out of range

In [2]:
print(''.join([i2w_brackets[i] for i in  x_train_brackets[10_000]]) )

NameError: name 'x_train_brackets' is not defined

In [4]:
len(x_train_ndfa[-1])

98

In [5]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, emb_size, h, num_char, n_layers=1):
        super(LSTM, self).__init__()

        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.lstm = nn.LSTM(input_size=emb_size, hidden_size=h, num_layers=n_layers, batch_first=True)
        self.fc = nn.Linear(h, num_char)
    
    def forward(self, input_seq, h):

        embedded = self.embedding(input_seq)
        lstm_out, hidden = self.lstm(embedded, h)
        output = self.fc(lstm_out)
        return output, hidden


In [6]:
x_train_ndfa = pad_and_convert(x_train_ndfa)
x_train_brackets = pad_and_convert(x_train_brackets)

In [78]:
print(type(x_train_ndfa))

<class 'torch.Tensor'>


In [7]:
print(x_train_ndfa.size())

torch.Size([1000, 98])


In [12]:
model = LSTM(vocab_size=len(set(w2i_ndfa)), emb_size=300, h=300, num_char=len(set(w2i_ndfa)), n_layers=1)

In [13]:
vocab_size = len(set(w2i_ndfa))
emb_size = 300
h = 300
num_char = vocab_size
n_layers = 1

In [14]:
num_epochs = 1
learning_rate = 0.001

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
x_train_ndfa_tensor = torch.tensor(x_train_ndfa)

C:\Users\casou\AppData\Local\Temp\ipykernel_17332\4158342828.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train_ndfa_tensor = torch.tensor(x_train_ndfa)


In [24]:
for epoch in range(num_epochs):
    total_loss = 0.0

    # Use DataLoader for batching
    dataset = TensorDataset(x_train_ndfa_tensor)
    dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

    for batch_idx, (inputs,) in enumerate(dataloader):
        print(f'Batch Index: {batch_idx}, Batch Size: {inputs.size(0)}')        
        optimizer.zero_grad()

        # Autoregressive training
        h = None
        for t in range(inputs.size(1) - 1):  # -1 to exclude the last time step
            input_seq = inputs[:, :t + 1]  # Input sequence up to time step t
            target_word = inputs[:, t + 1]  # Target is the next word (shifted by one time step)

            # Forward pass
            output, _ = model(input_seq, h)

            # Compute the loss
            loss = criterion(output[:, -1, :], target_word)  # Only consider the last time step

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Print the current iteration (outside the inner loop)
        print(f'Epoch [{epoch+1}/{num_epochs}], Iteration [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

    average_loss = total_loss / len(dataloader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss:.4f}')


Batch Index: 0, Batch Size: 10
Epoch [1/1], Iteration [1/100], Loss: 0.0000
Batch Index: 1, Batch Size: 10
Epoch [1/1], Iteration [2/100], Loss: 0.0000
Batch Index: 2, Batch Size: 10


KeyboardInterrupt: 

In [30]:
# Convert to list and print
print(list(enumerate(dataloader))[:2])




[(0, [tensor([[ 9, 14,  6,  7, 12, 14,  6,  7, 12, 14,  6,  7, 12,  9,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 9, 13,  4, 11, 12,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 9,  8,  5, 10, 12,  8,  5, 10, 12,  8,  5, 10, 12,  8,  5, 10, 12,  8,
          5, 10, 12

In [15]:
print(model)

LSTM(
  (embedding): Embedding(15, 300)
  (lstm): LSTM(300, 300, batch_first=True)
  (fc): Linear(in_features=300, out_features=15, bias=True)
)
